# Importação das bibliotecas e Criação dos dataframes

In [2]:
import pandas as pd
import sys
sys.path.append('../')

pd.set_option('display.float_format', lambda x: '%0f' % x)

df_energy = pd.read_csv('../datasets/raw/consumo_energia_eletrica.csv', sep=',')
df_region = pd.read_csv('../datasets/raw/estado_regiao.csv', sep=';')

## Analise inicial dos dados de consumo

In [3]:
df_energy.head()

,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo
0,2004,1,TO,Total,NaN,65876
1,2004,1,BA,Total,NaN,1444451
2,2004,1,PR,Total,NaN,1596274
3,2004,1,RS,Total,NaN,1780912
4,2004,1,GO,Total,NaN,630624


In [4]:
df_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39897 entries, 0 to 39896
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  39897 non-null  int64  
 1   mes                  39897 non-null  int64  
 2   sigla_uf             39897 non-null  object 
 3   tipo_consumo         39897 non-null  object 
 4   numero_consumidores  26937 non-null  float64
 5   consumo              39897 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 1.8+ MB


A partir da analise inicial dos dados que alguns formatos não correspondem ao que seria esperado para natureza do field. \
Como por exemplo, o field numero_consumidores que é apresentado incialmente como float64. Também foi possível identificar a presença de valores nulos no mesmo field.

In [5]:
df_energy.duplicated().sum()


np.int64(1017)

Foram identificadas 1017 linhas duplicadas a serem removidas do dataframe 

In [6]:
df_energy.drop_duplicates(inplace=True)

In [7]:
df_energy.describe().round(2)

,ano,mes,numero_consumidores,consumo
count,38880.000000,38880.000000,25920.000000,38880.000000
mean,2013.500000,6.500000,682303.380000,611732.010000
std,5.770000,3.450000,1839311.690000,1208066.850000
min,2004.000000,1.000000,52.000000,354.000000
25%,2008.750000,3.750000,18956.500000,85476.000000
50%,2013.500000,6.500000,108174.500000,221062.000000
75%,2018.250000,9.250000,442412.000000,596699.000000
max,2023.000000,12.000000,19201264.000000,12809742.000000


Ações de correção na base: 
 1. Alterar o tipo da coluna número consumidores de float para int
 2. Alterar o tipo da coluna consumo de int para float 
 3. Tratar os dados nulos da coluna número consumidores 
 4. Tratar possíveis outliers da coluna consumo


In [8]:
df_energy_custom = df_energy
df_energy_custom['consumo'] = df_energy_custom['consumo'].astype('float')
df_energy_custom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38880 entries, 0 to 38879
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  38880 non-null  int64  
 1   mes                  38880 non-null  int64  
 2   sigla_uf             38880 non-null  object 
 3   tipo_consumo         38880 non-null  object 
 4   numero_consumidores  25920 non-null  float64
 5   consumo              38880 non-null  float64
dtypes: float64(2), int64(2), object(2)
memory usage: 2.1+ MB


In [9]:

df_energy_custom.groupby(['sigla_uf','tipo_consumo'])['numero_consumidores'].sum().reset_index()

,sigla_uf,tipo_consumo,numero_consumidores
0,AC,Cativo,0.000000
1,AC,Comercial,4376707.000000
2,AC,Industrial,140290.000000
3,AC,Outros,6353093.000000
4,AC,Residencial,42270581.000000
...,...,...,...
157,TO,Comercial,7888898.000000
158,TO,Industrial,480289.000000
159,TO,Outros,14157539.000000
160,TO,Residencial,96462459.000000


In [10]:
df_energy_custom.isnull().sum()

ano                        0
mes                        0
sigla_uf                   0
tipo_consumo               0
numero_consumidores    12960
consumo                    0
dtype: int64

Agrupamos o número de consumidores por tipo de consumo para tentar identificar onde haveria maior concentração de valores nulos.

In [11]:
df_energy_custom.groupby(['tipo_consumo'])['numero_consumidores'].sum().reset_index()

,tipo_consumo,numero_consumidores
0,Cativo,0.000000
1,Comercial,1264142056.000000
2,Industrial,125271021.000000
3,Outros,1110354277.000000
4,Residencial,15185536183.000000
5,Total,0.000000


Foi possível identificar que os tipos de consumo 'Cativo' e 'Total' representam todos os valores nulos identificados

In [12]:
df_energy_custom.groupby(['tipo_consumo'])['consumo'].sum().reset_index()

,tipo_consumo,consumo
0,Cativo,6201517579.000000
1,Comercial,1547394950.000000
2,Industrial,3463509299.000000
3,Outros,1359081971.000000
4,Residencial,2421323423.000000
5,Total,8791313198.000000


A partir dessa constatação levantamos a suspeita de que o tipo de consumo 'total' representava o somatório dos demais tipos não nulos. \
Para testar a hipótese elaboramos os filtros abaixo para comparar o total com o somatório dos tipos 'Comercial', 'Industrial', 'Outros', 'Residencial'.

In [13]:
filter = df_energy_custom['tipo_consumo'].isin(['Comercial', 'Industrial', 'Outros', 'Residencial'])
expenditure_filtered = df_energy_custom[filter]['consumo'].sum()
expenditure_total_type = df_energy_custom[df_energy_custom['tipo_consumo'] == 'Total']['consumo'].sum()


print(f'Total dos tipos sem Cativo e total {expenditure_filtered} e Total Geral {expenditure_total_type}')

Total dos tipos sem Cativo e total 8791309643.0 e Total Geral 8791313198.0


O Teste demonstrou que de fato existe a correlação levantada em hipótese. \
Dessa forma concluímos que o tipo de consumo total exibe dados redundantes e pode ser descartado do dataframe.

Pela mesma lógica, não foi possível identificar nenhuma combinação de somas dos tipos de consumo não nulos que resultasse no valor do tipo 'cativo'. \
Sendo assim é possível assumir que seu valor de consumo não possui relevancia no valor total obtido, e portanto, também pode ser removido do dataframe.

In [14]:
df_energy_custom.dropna(inplace=True)
df_energy_custom.groupby(['tipo_consumo'])['consumo'].sum()

tipo_consumo
Comercial     1547394950.000000
Industrial    3463509299.000000
Outros        1359081971.000000
Residencial   2421323423.000000
Name: consumo, dtype: float64

In [15]:
df_energy_custom['numero_consumidores'] = df_energy_custom['numero_consumidores'].astype('int')
df_energy_custom.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25920 entries, 648 to 38879
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  25920 non-null  int64  
 1   mes                  25920 non-null  int64  
 2   sigla_uf             25920 non-null  object 
 3   tipo_consumo         25920 non-null  object 
 4   numero_consumidores  25920 non-null  int64  
 5   consumo              25920 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.4+ MB


In [16]:
df_energy_custom.describe()

,ano,mes,numero_consumidores,consumo
count,25920.000000,25920.000000,25920.000000,25920.000000
mean,2013.500000,6.500000,682303.377199,339170.896721
std,5.766393,3.452119,1839311.687366,603642.745612
min,2004.000000,1.000000,52.000000,354.000000
25%,2008.750000,3.750000,18956.500000,61545.500000
50%,2013.500000,6.500000,108174.500000,136754.500000
75%,2018.250000,9.250000,442412.000000,361651.750000
max,2023.000000,12.000000,19201264.000000,4849895.000000


## Iniciando analise da base de regiões

In [17]:
df_region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id_estado  27 non-null     int64 
 1   sigla      27 non-null     object
 2   estado     27 non-null     object
 3   regiao     27 non-null     object
 4   pais       27 non-null     object
dtypes: int64(1), object(4)
memory usage: 1.2+ KB


In [18]:
df_region.head(10)

,id_estado,sigla,estado,regiao,pais
0,1,AC,Acre,Norte,Brasil
1,2,AL,Alagoas,Nordeste,Brasil
2,3,AP,Amapa,Norte,Brasil
3,4,AM,Amazonas,Norte,Brasil
4,5,BA,Bahia,Nordeste,Brasil
5,6,CE,Ceara,Nordeste,Brasil
6,7,DF,Distrito Federal,Centro-Oeste,Brasil
7,8,ES,Espirito Santo,Sudeste,Brasil
8,9,GO,Goias,Centro-Oeste,Brasil
9,10,MA,Maranhao,Nordeste,Brasil


Concluímos que não há necessidade de saneamento do dataset de Regiões e portanto podemos realizar o merge com o dataframe de consumo de energia visando obter dados por região.

In [19]:
df_merged_states = pd.merge(left=df_energy_custom, right=df_region, left_on='sigla_uf', right_on='sigla', how='left' )
df_merged_states.head()

,ano,mes,sigla_uf,tipo_consumo,numero_consumidores,consumo,id_estado,sigla,estado,regiao,pais
0,2004,1,RN,Outros,40857,69617.000000,20,RN,Rio Grande do Norte,Nordeste,Brasil
1,2004,1,SP,Outros,311650,937538.000000,25,SP,Sao Paulo,Sudeste,Brasil
2,2004,1,MS,Outros,56881,67601.000000,12,MS,Mato Grosso do Sul,Centro-Oeste,Brasil
3,2004,1,SC,Outros,226165,209380.000000,24,SC,Santa Catarina,Sul,Brasil
4,2004,1,RJ,Outros,70634,416128.000000,19,RJ,Rio de Janeiro,Sudeste,Brasil


In [20]:
df_merged_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25920 entries, 0 to 25919
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ano                  25920 non-null  int64  
 1   mes                  25920 non-null  int64  
 2   sigla_uf             25920 non-null  object 
 3   tipo_consumo         25920 non-null  object 
 4   numero_consumidores  25920 non-null  int64  
 5   consumo              25920 non-null  float64
 6   id_estado            25920 non-null  int64  
 7   sigla                25920 non-null  object 
 8   estado               25920 non-null  object 
 9   regiao               25920 non-null  object 
 10  pais                 25920 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 2.2+ MB


In [21]:
df_merged_states.groupby(['regiao'])['consumo'].sum().sort_values().reset_index()

,regiao,consumo
0,Norte,596647389.000000
1,Centro-Oeste,626086467.000000
2,Nordeste,1494826811.000000
3,Sul,1555233899.000000
4,Sudeste,4518515077.000000


In [22]:
df_merged_states.groupby(['regiao'])['numero_consumidores'].sum().sort_values().reset_index()

,regiao,numero_consumidores
0,Norte,1072648472
1,Centro-Oeste,1359056908
2,Sul,2676182474
3,Nordeste,4598745182
4,Sudeste,7978670501


Realizamos a criação de uma coluna data a partir das colunas ano e mes para que seja possível realizar filtros agrupados por mês e ano

In [23]:
df_merged_states['data'] = pd.to_datetime(df_merged_states['ano'].astype(str) + '-' + df_merged_states['mes'].astype(str) + '-01')

In [24]:

df_merged_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25920 entries, 0 to 25919
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ano                  25920 non-null  int64         
 1   mes                  25920 non-null  int64         
 2   sigla_uf             25920 non-null  object        
 3   tipo_consumo         25920 non-null  object        
 4   numero_consumidores  25920 non-null  int64         
 5   consumo              25920 non-null  float64       
 6   id_estado            25920 non-null  int64         
 7   sigla                25920 non-null  object        
 8   estado               25920 non-null  object        
 9   regiao               25920 non-null  object        
 10  pais                 25920 non-null  object        
 11  data                 25920 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 2.4+ MB


Realizamos a criação de uma coluna dataStr a partir da coluna data para visualização do dado em texto

In [25]:
df_merged_states['dataStr'] = df_merged_states['data'].dt.strftime('%B/%Y')


Apagamos as colunas 'sigla_uf' e 'id_estado' pois as informações não seriam importantes para análise proposta

In [26]:
df_merged_states.drop(columns=['sigla_uf', 'id_estado'], inplace=True)

In [27]:
df_merged_states.head()

,ano,mes,tipo_consumo,numero_consumidores,consumo,sigla,estado,regiao,pais,data,dataStr
0,2004,1,Outros,40857,69617.000000,RN,Rio Grande do Norte,Nordeste,Brasil,2004-01-01,January/2004
1,2004,1,Outros,311650,937538.000000,SP,Sao Paulo,Sudeste,Brasil,2004-01-01,January/2004
2,2004,1,Outros,56881,67601.000000,MS,Mato Grosso do Sul,Centro-Oeste,Brasil,2004-01-01,January/2004
3,2004,1,Outros,226165,209380.000000,SC,Santa Catarina,Sul,Brasil,2004-01-01,January/2004
4,2004,1,Outros,70634,416128.000000,RJ,Rio de Janeiro,Sudeste,Brasil,2004-01-01,January/2004


In [28]:
df_merged_states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25920 entries, 0 to 25919
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ano                  25920 non-null  int64         
 1   mes                  25920 non-null  int64         
 2   tipo_consumo         25920 non-null  object        
 3   numero_consumidores  25920 non-null  int64         
 4   consumo              25920 non-null  float64       
 5   sigla                25920 non-null  object        
 6   estado               25920 non-null  object        
 7   regiao               25920 non-null  object        
 8   pais                 25920 non-null  object        
 9   data                 25920 non-null  datetime64[ns]
 10  dataStr              25920 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 2.2+ MB


# Consideramos concluída à análise exploratória dos dados 

In [29]:
df_merged_states.to_csv('../datasets/processed/energy_by_region.csv', index=False)